In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, TimeDistributed
from keras.layers.normalization import BatchNormalization
from tqdm import tqdm_notebook, tqdm
from IPython.display import clear_output, display
from keras import backend as K

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, RobustScaler, Normalizer, MinMaxScaler
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier

from scipy import stats
from glob import glob
from multiprocessing import Pool
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import StratifiedKFold
import xgboost

Using Theano backend.
/home/billion/miniconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
Xshape = pd.read_csv('/mnt/kaspersky/data_kasp/train/0_train.csv', header=None).shape
Xshape

(96000, 56)

In [4]:
steps_ahead = 10
batch_size, max_len = 1, Xshape[0]

model = Sequential()
model.add(LSTM(16, batch_input_shape=(batch_size, max_len, Xshape[1]-1), return_sequences=True,
         ))
model.add(TimeDistributed(Dense(16, activation='relu')))
model.add(TimeDistributed(Dense(Xshape[1]-1, activation='linear')))
model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mse'])

In [5]:
w = keras.models.load_model('tmp/lstm1').weights
w = [i.get_value() for i in w]

model.set_weights(w)

In [ ]:
for name, data in ('train', pd.read_csv('train_labels.csv')), ('test', pd.read_csv('sample_submission.csv')):
    files = ['/mnt/kaspersky/data_kasp/'+name+'/'+f for f in data.SeriesId][::1]

    for f in tqdm(files):
        tmp = pd.read_csv(f, header=None).iloc[:,1:]
        tmp = MinMaxScaler().fit_transform(tmp)
        pr = model.predict(tmp[None,:,:])[0]

        functions = [np.min, np.max, np.mean, np.std]
        fts = np.stack([func((pr[:-10,:] - tmp[10:,:]) ** 2, axis=0) for func in functions]).ravel()
        np.save('tmp/lstmfeats/'+f.split('.')[0].split('/')[-1]+'.npy', fts)